# Setting up my new Dessert Shop

### 1. Business Problem
I am a local entrepreneur, looking to open my latest outlet in Singapore to sell my famous Salted egg cookies.

In order to determine where to open my store, i'll use data from FourSquare to determine the best location. The data i'll be using is the general category of venues for each neighbourhood.

### 2. Methodology
First we'll download the data to visualise the neighbourhoods in Singapore.

Next, we'll take a look into the profile of each neighbourhood, focusing on the Central Region. By kNN clustering analysis, we will look for patterns, and decide if which neighbourhoods are suitable for our operations.

### 3. Installing libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [2]:
#information to use FourSquare API
CLIENT_ID = 'EK1REWEV5DX3ZQYQS2WIE52WGLN5JPLXUDQN0CGWWFM2OMPO' # your Foursquare ID
CLIENT_SECRET = 'KX5UIJSX42SW4DSBXWSEYZKFOC0T4WCGOJUPIBDY01SMFDR0' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

### 4. Downloading Our Dataset

The data we will use to find our neighbourhood data is Singapore's Master Plan 2019, which divides Singapore into 54 neighbourhoods.

It contains a list of the boundaries of every neighbourhood.

Thus, we will use the mean average to find the centre of each neighbourhood, and save the information in a DataFrame called 'neighbourhoods'.

In [3]:
with open('C:/Users/User/Downloads/mygeodata/planning-boundary-area.geojson') as json_data:
    singapore_data = json.load(json_data)
singapore_data

{'type': 'FeatureCollection',
 'name': 'URA_MP19_PLNG_AREA_PL',
 'crs': {'type': 'name',
  'properties': {'name': 'urn:ogc:def:crs:OGC:1.3:CRS84'}},
 'features': [{'type': 'Feature',
   'properties': {'Name': 'kml_3',
    'description': None,
    'PLN_AREA_N': 'BUKIT TIMAH',
    'PLN_AREA_C': 'BT',
    'CA_IND': 'N',
    'REGION_N': 'CENTRAL REGION',
    'REGION_C': 'CR',
    'INC_CRC': '6CCDADD1F85173E9',
    'FMEL_UPD_D': '20191206144714',
    'Field_6': '6CCDADD1F85173E9',
    'Field_7': '20191206144714',
    'Field_4': 'CENTRAL REGION',
    'Field_5': 'CR',
    'Field_2': 'BT',
    'Field_3': 'N',
    'Field_1': 'BUKIT TIMAH'},
   'geometry': {'type': 'Polygon',
    'coordinates': [[[103.797658114828, 1.34812847309287],
      [103.798064914078, 1.34778923664106],
      [103.798495545895, 1.34737825091054],
      [103.799663605446, 1.34612978708951],
      [103.801363468782, 1.34434969463422],
      [103.80372566348, 1.34184555324235],
      [103.804213111113, 1.34135245083376],
   

In [4]:
neighbourhood_data = singapore_data['features']

In [5]:
neighbourhood_data[54]

{'type': 'Feature',
 'properties': {'Name': 'kml_55',
  'description': None,
  'PLN_AREA_N': 'YISHUN',
  'PLN_AREA_C': 'YS',
  'CA_IND': 'N',
  'REGION_N': 'NORTH REGION',
  'REGION_C': 'NR',
  'INC_CRC': 'D20FD597C82D190A',
  'FMEL_UPD_D': '20191206144714',
  'Field_6': 'D20FD597C82D190A',
  'Field_7': '20191206144714',
  'Field_4': 'NORTH REGION',
  'Field_5': 'NR',
  'Field_2': 'YS',
  'Field_3': 'N',
  'Field_1': 'YISHUN'},
 'geometry': {'type': 'Polygon',
  'coordinates': [[[103.859547616067, 1.42552363181964],
    [103.861970078945, 1.42447830842016],
    [103.863855079442, 1.42625923942526],
    [103.864060269135, 1.42645310060663],
    [103.864267287519, 1.42664868907847],
    [103.867421871155, 1.42498336067788],
    [103.867047814049, 1.42463561214612],
    [103.865118251817, 1.42301580763938],
    [103.864383816003, 1.42228475085299],
    [103.864290267711, 1.42217491261826],
    [103.864220758065, 1.42209928613661],
    [103.864156884356, 1.42202176945673],
    [103.8641174

In [6]:
# define the useful dataframe columns
column_names = ['Neighbourhood','Region','Latitude', 'Longitude'] 

# instantiate the dataframe
neighbourhoods = pd.DataFrame(columns=column_names)
neighbourhoods

,Neighbourhood,Region,Latitude,Longitude


In [7]:
for data in neighbourhood_data:
    neighbourhood_name = data['properties']['PLN_AREA_N']
    neighbourhood_region = data['properties']['REGION_N']
    #taking the mean of the boundary coordinates
    neighbourhood_latlon = np.mean(np.array(data['geometry']['coordinates']),axis=1)
    neighbourhood_lat = neighbourhood_latlon[0][1]
    neighbourhood_lon = neighbourhood_latlon[0][0]
    neighbourhoods = neighbourhoods.append({'Neighbourhood':neighbourhood_name,
                                            'Region':neighbourhood_region,
                                            'Latitude': neighbourhood_lat,
                                          'Longitude': neighbourhood_lon}, ignore_index=True)

Now lets observe how our DataFrame looks like.

In [8]:
neighbourhoods

,Neighbourhood,Region,Latitude,Longitude
0,BUKIT TIMAH,CENTRAL REGION,1.323922,103.791861
1,CENTRAL WATER CATCHMENT,NORTH REGION,1.364764,103.816642
2,CHOA CHU KANG,WEST REGION,1.389628,103.748133
3,NORTH-EASTERN ISLANDS,NORTH-EAST REGION,1.399500,103.986528
4,TENGAH,WEST REGION,1.360918,103.731075
5,PUNGGOL,NORTH-EAST REGION,1.404409,103.904620
6,BOON LAY,WEST REGION,1.316895,103.707595
7,MARINA EAST,CENTRAL REGION,1.290463,103.871004
8,PIONEER,WEST REGION,1.320819,103.673753
9,SEMBAWANG,NORTH REGION,1.446958,103.818263


Is there an easier way of representing this data visually?

### 5. Creating a map of Singapore

In [9]:
#coordinates of Singapore
latitude = 1.35
longitude = 103.833333

map_sg = folium.Map(location=[latitude, longitude], zoom_start=11.2)

# add markers to map
for lat, lon, region, neighbourhood in zip(neighbourhoods['Latitude'], neighbourhoods['Longitude'], neighbourhoods['Region'], neighbourhoods['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, region)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=6,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_sg)  
    
map_sg

### 6. Exploring a Neighbourhood

I want to open my shop in the Central Regioe. The Central Region of Singapore has more commercial and entertainment venues, rather than residential venues. Thus our target demographic would more likely be captured in the Central Region.

What are the neighbourhoods in this region like?

Lets explore the neighbourhoods in the Central Region in greater detail.

In [10]:
filter = neighbourhoods['Region'] == 'CENTRAL REGION'
neighbourhoods_central = neighbourhoods[filter]

In [11]:
neighbourhoods_central

,Neighbourhood,Region,Latitude,Longitude
0,BUKIT TIMAH,CENTRAL REGION,1.323922,103.791861
7,MARINA EAST,CENTRAL REGION,1.290463,103.871004
11,QUEENSTOWN,CENTRAL REGION,1.302038,103.790838
13,SOUTHERN ISLANDS,CENTRAL REGION,1.234221,103.830998
14,MARINA SOUTH,CENTRAL REGION,1.283023,103.864299
16,TANGLIN,CENTRAL REGION,1.305479,103.821713
19,DOWNTOWN CORE,CENTRAL REGION,1.286231,103.853380
20,NEWTON,CENTRAL REGION,1.306618,103.837161
21,ORCHARD,CENTRAL REGION,1.304055,103.833283
22,STRAITS VIEW,CENTRAL REGION,1.263193,103.855495


In [12]:
print ('There are ' + str(len(neighbourhoods_central)) + ' neighbourhoods in the Central Region.')

There are 22 neighbourhoods in the Central Region.


#### What is the profile of the Museum Neighbourhood(45)? Lets find out.

In [13]:
CLIENT_ID = 'EK1REWEV5DX3ZQYQS2WIE52WGLN5JPLXUDQN0CGWWFM2OMPO' # your Foursquare ID
CLIENT_SECRET = 'KX5UIJSX42SW4DSBXWSEYZKFOC0T4WCGOJUPIBDY01SMFDR0' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentials:
CLIENT_ID: EK1REWEV5DX3ZQYQS2WIE52WGLN5JPLXUDQN0CGWWFM2OMPO
CLIENT_SECRET:KX5UIJSX42SW4DSBXWSEYZKFOC0T4WCGOJUPIBDY01SMFDR0


In [14]:
LIMIT = 100
radius = 500
neighbourhood_latitude = 1.297895
neighbourhood_longitude = 103.847149
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighbourhood_latitude, 
    neighbourhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=EK1REWEV5DX3ZQYQS2WIE52WGLN5JPLXUDQN0CGWWFM2OMPO&client_secret=KX5UIJSX42SW4DSBXWSEYZKFOC0T4WCGOJUPIBDY01SMFDR0&v=20180605&ll=1.297895,103.847149&radius=500&limit=100'

In [15]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5edc38a060ba08001b501bc5'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Central Region',
  'headerFullLocation': 'Central Region, Singapore',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 90,
  'suggestedBounds': {'ne': {'lat': 1.3023950045000046,
    'lng': 103.85164175512823},
   'sw': {'lat': 1.2933949954999955, 'lng': 103.84265624487178}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '53a2da5d498edd040c594591',
       'name': 'The Grand Cathay',
       'location': {'address': 'Cathay Cineplex',
        'crossStreet': '2 Handy Rd.',
        'lat': 1.2992198664952235,
        'lng': 103.84791634393056,
        'labeled

In [16]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [17]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(10)

C:\Users\User\anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,The Grand Cathay,Movie Theater,1.299220,103.847916
1,The Auld Alliance,Whisky Bar,1.298567,103.848903
2,National Museum of Singapore,Museum,1.296498,103.848462
3,KOI Thé,Bubble Tea Shop,1.299208,103.845523
4,Yoga Movement,Yoga Studio,1.299164,103.846912
5,Fort Canning Park,Park,1.295075,103.846421
6,P.Osh,Dessert Shop,1.298832,103.845811
7,St. Games,Gaming Cafe,1.299040,103.847737
8,Singapore History Gallery,History Museum,1.296483,103.848410
9,The Tea Party,Bubble Tea Shop,1.297671,103.848808


Not surprisingly, Koi made it into our top 5 list. Bubble tea addicts.

How many venues were returned by FourSquare?

In [18]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

90 venues were returned by Foursquare.


In [19]:
nearby_venues.groupby('categories').count()

,name,lat,lng
categories,,,
Accessories Store,1,1,1
Arcade,1,1,1
Art Gallery,1,1,1
Art Museum,1,1,1
Arts & Crafts Store,1,1,1
Asian Restaurant,3,3,3
Bar,1,1,1
Bubble Tea Shop,2,2,2
Bus Station,1,1,1


Just from a quick glance, F&B options are quite popular in the Museum District.

It'll probably be a good place to set up a restaurant, as people will travel here to explore dining options.

### 7. Lost in Central

Now you might be wondering:

But what about the rest of the Central Area? What is popular?

No worries!

Let's create a function to repeat the same process to all the neighborhoods in the Central Region.

In [20]:
def getNearbyVenues(names, lat, lon, radius=1000, limit = 100):
     venues_list=[]
     for name, lat, lon in zip(names, lat, lon):
        print(name)
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION,
            lat,
            lon,
            radius, 
            limit)
                
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']

        # return only relevant information for each nearby venue
        venues_list.append([(
            name,  
            v['venue']['name'], 
            v['venue']['categories'][0]['name']) for v in results])

     nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
     nearby_venues.columns = ['Neighbourhood', 'Venue', 'Venue Category']
     
     return(nearby_venues)

In [21]:
central_venues = getNearbyVenues(names=neighbourhoods_central['Neighbourhood'],
                                 lat=neighbourhoods_central['Latitude'],
                                 lon=neighbourhoods_central['Longitude'])

BUKIT TIMAH
MARINA EAST
QUEENSTOWN
SOUTHERN ISLANDS
MARINA SOUTH
TANGLIN
DOWNTOWN CORE
NEWTON
ORCHARD
STRAITS VIEW
BUKIT MERAH
KALLANG
NOVENA
BISHAN
TOA PAYOH
MUSEUM
OUTRAM
RIVER VALLEY
ROCHOR
SINGAPORE RIVER
MARINE PARADE
GEYLANG


See any familiar names? #Throwback to pre-Covid times where we could go shopping rite?

In [22]:
central_venues

,Neighbourhood,Venue,Venue Category
0,BUKIT TIMAH,Plank Sourdough Pizza By Baker & Cook,Pizza Place
1,BUKIT TIMAH,Simply Bread,Bakery
2,BUKIT TIMAH,Brazil Churrasco,Churrascaria
3,BUKIT TIMAH,MEAT n' CHILL,BBQ Joint
4,BUKIT TIMAH,Caruso Ristorante Pizza Bar,Pizza Place
5,BUKIT TIMAH,Baker & Cook,Bakery
6,BUKIT TIMAH,Ikeda Spa,Spa
7,BUKIT TIMAH,Cold Storage,Supermarket
8,BUKIT TIMAH,Delifrance,Café
9,BUKIT TIMAH,The Tea Party Café,Tea Room


How many listed venues are there for each Neighbourhood?

(Remember we set our venue limit per neighbourhood to 100 in the previous section.)

In [23]:
central_venues.groupby('Neighbourhood').count()

,Venue,Venue Category
Neighbourhood,,
BISHAN,84,84
BUKIT MERAH,93,93
BUKIT TIMAH,49,49
DOWNTOWN CORE,100,100
GEYLANG,100,100
KALLANG,100,100
MARINA EAST,61,61
MARINA SOUTH,96,96
MARINE PARADE,55,55


#### Let's find out how many unique categories can be curated from all the returned venues

In [24]:
print('There are {} uniques categories.'.format(len(central_venues['Venue Category'].unique())))

There are 230 uniques categories.


In [25]:
central_venues['Venue Category'].unique()

array(['Pizza Place', 'Bakery', 'Churrascaria', 'BBQ Joint', 'Spa',
       'Supermarket', 'Café', 'Tea Room', 'Indian Restaurant',
       'Restaurant', 'Bar', 'Coffee Shop', 'Japanese Restaurant',
       'Bookstore', 'Beer Garden', 'Shopping Mall', 'Seafood Restaurant',
       'Wine Bar', 'Food Court', 'Bus Station', 'Metro Station',
       'Italian Restaurant', 'Building', 'Park', 'Flower Shop',
       'Sporting Goods Shop', 'Bank', 'Garden', 'Canal', 'Pet Store',
       'Pharmacy', 'Toy / Game Store', 'Golf Driving Range',
       'Golf Course', 'Event Space', 'Theme Park Ride / Attraction',
       'Racetrack', 'Playground', 'Waterfront', 'Scenic Lookout',
       'Botanical Garden', 'Bridge', 'American Restaurant', 'Art Gallery',
       'Movie Theater', 'French Restaurant', 'Lounge', 'Snack Place',
       'Satay Restaurant', 'Sandwich Place', 'Theme Park',
       'Tourist Information Center', 'Ice Cream Shop',
       'Athletics & Sports', 'Gift Shop', 'Cafeteria',
       'Chinese Rest

Cool. Now we know what we are looking at. Can we information to determine to hottest venues in each neighbourhood?

In [26]:
# one hot encoding
central_onehot = pd.get_dummies(central_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
central_onehot['Neighbourhood'] = central_venues['Neighbourhood']
fixed_columns = [central_onehot.columns[-1]] + list(central_onehot.columns[:-1])
central_onehot = central_onehot[fixed_columns]
central_onehot.head(20)

,Neighbourhood,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Basketball Court,Bay,Beach,Beer Bar,Beer Garden,Bistro,Board Shop,Boarding House,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buddhist Temple,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Cafeteria,Café,Canal,Cantonese Restaurant,Casino,Chinese Restaurant,Chocolate Shop,Church,Churrascaria,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Cruise,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Discount Store,Dive Bar,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,English Restaurant,Event Space,Exhibit,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fujian Restaurant,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,General College & University,General Entertainment,German Restaurant,Gift Shop,Golf Course,Golf Driving Range,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hainan Restaurant,Halal Restaurant,Harbor / Marina,Historic Site,History Museum,Hong Kong Restaurant,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Island,Italian Restaurant,Japanese Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Lighthouse,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoor Sculpture,Outdoors & Recreation,Paper / Office Supplies Store,Park,Pedestrian Plaza,Peking Duck Restaurant,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pie Shop,Pier,Pizza Place,Playground,Plaza,Pool,Pop-Up Shop,Pub,Racetrack,Ramen Restaurant,Residential Building (Apartment / Condo),Resort,Restaurant,River,Road,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Satay Restaurant,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Snack Place,Soccer Field,Soccer Stadium,Soup Place,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Club,Stables,Stationery Store,Steakhouse,Street Food Gathering,Supermarket,Surf Spot,Sushi Restaurant,TV Station,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Tourist Information Center,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Water Park,Waterfront,Whisky Bar,Wine Bar,Yoga Studio
0,BUKIT TIMAH,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,BUKIT TIMAH,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [27]:
central_onehot.shape

(1714, 231)

In [28]:
central_grouped = central_onehot.groupby('Neighbourhood').mean().reset_index()
central_grouped

,Neighbourhood,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Basketball Court,Bay,Beach,Beer Bar,Beer Garden,Bistro,Board Shop,Boarding House,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buddhist Temple,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Cafeteria,Café,Canal,Cantonese Restaurant,Casino,Chinese Restaurant,Chocolate Shop,Church,Churrascaria,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Cruise,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Discount Store,Dive Bar,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,English Restaurant,Event Space,Exhibit,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fujian Restaurant,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,General College & University,General Entertainment,German Restaurant,Gift Shop,Golf Course,Golf Driving Range,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hainan Restaurant,Halal Restaurant,Harbor / Marina,Historic Site,History Museum,Hong Kong Restaurant,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Island,Italian Restaurant,Japanese Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Lighthouse,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoor Sculpture,Outdoors & Recreation,Paper / Office Supplies Store,Park,Pedestrian Plaza,Peking Duck Restaurant,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pie Shop,Pier,Pizza Place,Playground,Plaza,Pool,Pop-Up Shop,Pub,Racetrack,Ramen Restaurant,Residential Building (Apartment / Condo),Resort,Restaurant,River,Road,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Satay Restaurant,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Snack Place,Soccer Field,Soccer Stadium,Soup Place,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Club,Stables,Stationery Store,Steakhouse,Street Food Gathering,Supermarket,Surf Spot,Sushi Restaurant,TV Station,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Tourist Information Center,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Water Park,Waterfront,Whisky Bar,Wine Bar,Yoga Studio
0,BISHAN,0.000000,0.000000,0.000000,0.00,0.023810,0.000000,0.00,0.011905,0.000000,0.035714,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.011905,0.00,0.000000,0.023810,0.011905,0.00,0.000000,0.071429,0.000000,0.000000,0.000000,0.119048,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.047619,0.000000,0.000000,0.000000,0.000000,0.011905,0.000000,0.000000,0.000000,0.000000,0.023810,0.000000,0.011905,0.00,0.000000,0.00,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.071429,0.000000,0.00,0.011905,0.000000,0.011905,0.000000,0.000000,0.00,

In [29]:
num_top_venues = 10

for x in central_grouped['Neighbourhood']:
    print("----"+x+"----")
    temp = central_grouped[central_grouped['Neighbourhood'] == x].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----BISHAN----
                venue  freq
0  Chinese Restaurant  0.12
1          Food Court  0.07
2                Café  0.07
3     Thai Restaurant  0.06
4         Coffee Shop  0.05
5      Ice Cream Shop  0.05
6              Bakery  0.04
7   Indian Restaurant  0.04
8               Trail  0.02
9    Sushi Restaurant  0.02


----BUKIT MERAH----
                 venue  freq
0          Coffee Shop  0.09
1                 Café  0.09
2   Chinese Restaurant  0.08
3     Asian Restaurant  0.08
4   Seafood Restaurant  0.06
5                Hotel  0.04
6               Bakery  0.04
7  Japanese Restaurant  0.04
8           Food Court  0.03
9          Snack Place  0.03


----BUKIT TIMAH----
                venue  freq
0          Food Court  0.08
1         Bus Station  0.08
2              Bakery  0.08
3         Coffee Shop  0.06
4   Indian Restaurant  0.06
5         Pizza Place  0.04
6         Supermarket  0.04
7                Café  0.04
8  Italian Restaurant  0.04
9            Building  0.02


----

Note: As FourSquare is not as popular here in Singapore, there is limited information available.

As such, please ignore any venue categories with a frequency of 0.

So how can we use this to relate back to our original Question?

### 8. Clustering of neighbourhoods via kNN

In [30]:
# set number of clusters
kclusters = 7
central_grouped_clusters = central_grouped.drop('Neighbourhood', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(central_grouped_clusters)
kmeans.labels_[0:10]

array([6, 6, 0, 3, 6, 6, 3, 3, 6, 4])

In [31]:
# add clustering labels
central_grouped.insert(0, 'Cluster Labels', kmeans.labels_)

In [42]:
central_grouped.sort_values(['Cluster Labels'])

,Cluster Labels,Neighbourhood,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Basketball Court,Bay,Beach,Beer Bar,Beer Garden,Bistro,Board Shop,Boarding House,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buddhist Temple,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Cafeteria,Café,Canal,Cantonese Restaurant,Casino,Chinese Restaurant,Chocolate Shop,Church,Churrascaria,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Cruise,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Discount Store,Dive Bar,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,English Restaurant,Event Space,Exhibit,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fujian Restaurant,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,General College & University,General Entertainment,German Restaurant,Gift Shop,Golf Course,Golf Driving Range,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hainan Restaurant,Halal Restaurant,Harbor / Marina,Historic Site,History Museum,Hong Kong Restaurant,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Island,Italian Restaurant,Japanese Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Lighthouse,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoor Sculpture,Outdoors & Recreation,Paper / Office Supplies Store,Park,Pedestrian Plaza,Peking Duck Restaurant,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pie Shop,Pier,Pizza Place,Playground,Plaza,Pool,Pop-Up Shop,Pub,Racetrack,Ramen Restaurant,Residential Building (Apartment / Condo),Resort,Restaurant,River,Road,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Satay Restaurant,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Snack Place,Soccer Field,Soccer Stadium,Soup Place,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Club,Stables,Stationery Store,Steakhouse,Street Food Gathering,Supermarket,Surf Spot,Sushi Restaurant,TV Station,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Tourist Information Center,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Water Park,Waterfront,Whisky Bar,Wine Bar,Yoga Studio
2,0,BUKIT TIMAH,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.020408,0.000000,0.081633,0.020408,0.020408,0.000000,0.000000,0.000000,0.000000,0.020408,0.000000,0.000000,0.00,0.000000,0.020408,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.020408,0.000000,0.00,0.000000,0.081633,0.000000,0.00,0.000000,0.040816,0.020408,0.000000,0.000000,0.000000,0.000000,0.000000,0.020408,0.000000,0.000000,0.000000,0.061224,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.020408,0.000000,0.000000,0.081633,0.000000,0.00,0.000000,0.000000,0.000000,0

Any similarities between the clusters?
Cluster 0: Residential, few Rastaurants
Cluster 1(Straits View): Boat or Ferry (Marina South Ferry Terminal)
Cluster 2(Southern Islands): Boat or Ferry (Cruises to St Johns Island)
Cluster 3(Marina Bay): Event Spaces, Exhibitions (Marina Bay Sands), Parks(Marina Barrage)
Cluster 4(Orchard Rd): Hotels, Cafes and Restaurants
Cluster 5(River Valley): Hotels, Cafes and Restaurants
Cluster 6(Rest of Central): Restaurants, Chinese Restaurants, Food Court

### 9. Results and Analysis

Recall that we are an entrepeneur looking to open our new outlet (after CB). One of the factors we can consider is the predominant types of venues in that area.

Setting up shop with many restaurants might be advantageous, as people who just left the restaurants might come to our shop to pick up some Salted Egg treats. Meanwhile, having too many dessert shops might adversely affect sales due to the stiff competition.

By observing the cluster similarities, we can zoom in on urban neighbourhoods from Cluster 4 and Cluster 5, with a high density of F&B options such as Food Courts and various Restaurants.

At the same time, we can place a smaller focus on more residential neighbourhoods from Cluster 0 such as Bukit Timah, which consists of mainly private housing and schools. Not as ideal.

Lastly, we can forget about Straits View and Southern Islands Theres only a boat jetty.

### 10. Conclusion

From our findings, we can conclude that the best locations to set up shop would be in neighbourhoods from clusters 4 and 5, such as Orchard and River Valley. This is because these neighbourhoods are popular shopping destinations for both locals and tourists alike, thus foot traffic here will likely be the highest.

Other factors which we could explore in a subsequent analysis would include cost of rental, as well as customer segmentation and sampling to determine their taste and preferences.

Our Analysis of each neighbourhood is also limited by where we place our Geographic Centre of the neighbourhood. For example, the Southern Islands neighbourhood actually encompasses Sentosa, which being a tourist hotspot might be a good location for us, but falls outside our search radius.

Thus, further refinement of our model will be highly beneficial.